# web srabing + merge dataset  from kaggle


### web srabing 
1- import libraries 

2- request to get data form website

3- clean data

4- store data extacted from website in list

5- use pandas datafram to save it in csv file.

### merge dataset from kaggle
1- upload data from kaggle

2- clean data 

3- merge datafram1(data form website) to datafarme2(data form website) 


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [3]:
url = "https://www.imdb.com/search/title/?title_type=video_game&num_votes=,5000,&sort=user_rating,desc&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=87cca6a7-a16d-42d9-b9de-6aace99ec40a&pf_rd_r=ERFX36S273PQKZHMN3NF&pf_rd_s=center-6&pf_rd_t=60601&pf_rd_i=video-games&ref_=fea_vg_scg_ats_toprated_hd"

response = requests.get(url)
response.status_code

200

In [4]:
page = response.text

In [5]:
soup = BeautifulSoup(page, "lxml")

In [77]:
#soup.find('div',class_='lister-item mode-advanced')

In [7]:
title = soup.find('h3',class_="lister-item-header").a.text
title

'Wiedzmin 3: Dziki Gon Krew i wino'

In [62]:
year = soup.find('span',class_="lister-item-year text-muted unbold").text.split()[0].replace('(','').replace(')','')
year

'2016'

In [9]:
Gtype = soup.find('p',class_="text-muted").text.strip()
Gtype

'Action, Adventure, Drama'

In [10]:
rate = soup.find('div',class_="ratings-bar").find('strong').text
rate

'9.8'

In [11]:
descr = soup.find_all('p',class_="text-muted")[1].text.strip()
descr

"Geralt is in the southern province of Toussaint where a monstrous serial killer is targeting knights with a dark past. Geralt and his old vampire friend investigate the killer's motives."

In [12]:
votes = soup.find('p',class_='sort-num_votes-visible').text.split()[1].replace(',','')
votes

'6810'

In [13]:
def clean_num(s):
    """cleans text scraped from website
    
    Parameters:
    s (String): raw text

    Returns:
    String: cleaned digit text

    """ 

    if s is None:
        return s
    return "".join(re.findall('\d*\.?\d+',s))

def get_data(block, tag, class_text):
    """fetch data from tag and its associated class
    
    Parameters:
    block (int): Book item 
    tag (String): tag for element we want to fetch
    class_text (String): class for element we want to fetch

    Returns:
    String: data text fetched

    """ 
    try:
        return block.find(tag, class_ = class_text).text.replace('(','').replace(')','')
    except AttributeError:
        return ""
    return

def get_data_a_tag(i, tag, class_text):    
    try:
        return i.find(tag, class_ = class_text).a.text
    except AttributeError:
        return ""
    return
def get_data_strong_tag(i, tag, class_text):    
    try:
        return i.find(tag, class_ = class_text).strong.text
    except AttributeError:
        return ""
    return
def get_data_descr(i, tag, class_text):    
    try:
        return i.find_all(tag, class_ = class_text)[1].text.strip()
    except AttributeError:
        return ""
    return


def get_data_votes(i, tag, class_text):    
    try:
        return i.find(tag, class_ = class_text).text.split()[1].replace(',','')
    except AttributeError:
        return ""
    return

In [14]:
#Test with the first videio game to ensure functions are working
for i in (soup.find_all('div','lister-item mode-advanced')):
    print(get_data_a_tag(i, "h3","lister-item-header"))
    print(get_data(i, "span", "lister-item-year text-muted unbold"))
    print(get_data(i, "p", "text-muted").strip())
    print(clean_num(get_data_strong_tag(i, "div","ratings-bar")))
    print(get_data_descr(i, "p","text-muted"))
    print(clean_num(get_data_votes(i, "p","sort-num_votes-visible")))

Wiedzmin 3: Dziki Gon Krew i wino
2016 Video Game
Action, Adventure, Drama
9.8
Geralt is in the southern province of Toussaint where a monstrous serial killer is targeting knights with a dark past. Geralt and his old vampire friend investigate the killer's motives.
6810
Red Dead Redemption II
2018 Video Game
Action, Adventure, Crime
9.7
Amidst the decline of the Wild West at the turn of the 20th century, outlaw Arthur Morgan and his gang struggle to cope with the loss of their way of life.
28686
Wiedzmin 3: Dziki Gon
2015 Video Game
banned
|

Action, Adventure, Drama
9.7
A monster hunter for hire embarks on an epic journey to find his former apprentice before The Wild Hunt can capture her and bring about the destruction of the world.
23029
The Last of Us
2013 Video Game
Action, Adventure, Drama
9.7
In a hostile, post-pandemic world, Joel and Ellie, brought together by desperate circumstances, must rely on each other to survive a brutal journey across what remains of the United States.


Action, Adventure, Drama
9.3
Set in 1274 on the Tsushima Island, the last samurai, Jin Sakai, must master a new fighting style, the way of the Ghost, to defeat the Mongol forces and fight for the freedom and independence of Japan.
5962
Kingdom Hearts HD 1.5 + 2.5 Remix
2017 Video Game
Adventure, Family, Fantasy
9.3
A re-release of Kingdom Hearts HD 1.5 Remix and Kingdom Hearts HD 2.5 Remix on the Playstation 4.
590
Batman: Return to Arkham
2016 Video Game
Action, Adventure, Crime
9.3
Batman: Return to Arkham thrusts players back into the environments of Arkham Asylum and Arkham City with a remastered graphics engine and all DLC.
1415
Persona 5
2016 Video Game
Action, Adventure, Fantasy
9.3
A group of high school students create a vigilante group to expose the crimes of corrupt adults but soon find themselves in the midst of a worldwide conspiracy and targeted by powerful individuals who would do anything to silence them.
2333
Kingdom Hearts HD 2.5 Remix
2014 Video Game
Action, Adventur

In [15]:
    print(clean_num(get_data_votes(i, "p","sort-num_votes-visible")))

3830


In [63]:
Vgame_list = []

for i in (soup.find_all('div','lister-item mode-advanced')):
    Name = get_data_a_tag(i, "h3","lister-item-header")
    year = get_data(i, "span", "lister-item-year text-muted unbold").split()[0]
    Gtype = get_data(i, "p", "text-muted").strip()
    rate = clean_num(get_data_strong_tag(i, "div","ratings-bar"))
    votes = clean_num(get_data_votes(i, "p","sort-num_votes-visible"))
    
    headers = ['Name', 'year', 'game_type', 'rate','votes']
    Vgame_dict = dict(zip(headers, [ Name,
                                    year,
                                    Gtype,
                                    rate,
                                 
                                    votes]))
    Vgame_list.append(Vgame_dict)


In [64]:
    Vgame_list

[{'Name': 'Wiedzmin 3: Dziki Gon Krew i wino',
  'year': '2016',
  'game_type': 'Action, Adventure, Drama',
  'rate': '9.8',
  'votes': '6810'},
 {'Name': 'Red Dead Redemption II',
  'year': '2018',
  'game_type': 'Action, Adventure, Crime',
  'rate': '9.7',
  'votes': '28686'},
 {'Name': 'Wiedzmin 3: Dziki Gon',
  'year': '2015',
  'game_type': 'banned\n|\n\nAction, Adventure, Drama',
  'rate': '9.7',
  'votes': '23029'},
 {'Name': 'The Last of Us',
  'year': '2013',
  'game_type': 'Action, Adventure, Drama',
  'rate': '9.7',
  'votes': '55702'},
 {'Name': 'Mass Effect: Legendary Edition',
  'year': '2021',
  'game_type': 'Sci-Fi',
  'rate': '9.7',
  'votes': '706'},
 {'Name': 'God of War',
  'year': '2018',
  'game_type': 'Action, Adventure, Drama',
  'rate': '9.6',
  'votes': '21310'},
 {'Name': 'Perusona 5: Za roiyaru',
  'year': '2019',
  'game_type': 'Action, Adventure, Fantasy',
  'rate': '9.6',
  'votes': '798'},
 {'Name': 'Zeruda no densetsu: Toki no okarina',
  'year': '1998'

In [65]:
df = pd.DataFrame(Vgame_list)


In [66]:
df.Name=df.Name.str.lower()
df.head()

,Name,year,game_type,rate,votes
0,wiedzmin 3: dziki gon krew i wino,2016,"Action, Adventure, Drama",9.8,6810
1,red dead redemption ii,2018,"Action, Adventure, Crime",9.7,28686
2,wiedzmin 3: dziki gon,2015,"banned\n|\n\nAction, Adventure, Drama",9.7,23029
3,the last of us,2013,"Action, Adventure, Drama",9.7,55702
4,mass effect: legendary edition,2021,Sci-Fi,9.7,706


In [67]:
df.to_csv('video_game.csv')

In [68]:
df2=pd.read_csv('vgsales.csv')

In [86]:
df2.Name=df.Name.str.lower()
df2.head()

,Rank,Name,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,wiedzmin 3: dziki gon krew i wino,2006.0,41.49,29.02,3.77,8.46,82.74
1,2,red dead redemption ii,1985.0,29.08,3.58,6.81,0.77,40.24
2,3,wiedzmin 3: dziki gon,2008.0,15.85,12.88,3.79,3.31,35.82
3,4,the last of us,2009.0,15.75,11.01,3.28,2.96,33.00
4,5,mass effect: legendary edition,1996.0,11.27,8.89,10.22,1.00,31.37


In [87]:
Vgames=df.merge(df2,on='Name')
Vgames

,Name,year,game_type,rate,votes,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,wiedzmin 3: dziki gon krew i wino,2016,"Action, Adventure, Drama",9.8,6810,1,2006.0,41.49,29.02,3.77,8.46,82.74
1,red dead redemption ii,2018,"Action, Adventure, Crime",9.7,28686,2,1985.0,29.08,3.58,6.81,0.77,40.24
2,wiedzmin 3: dziki gon,2015,"banned\n|\n\nAction, Adventure, Drama",9.7,23029,3,2008.0,15.85,12.88,3.79,3.31,35.82
3,the last of us,2013,"Action, Adventure, Drama",9.7,55702,4,2009.0,15.75,11.01,3.28,2.96,33.00
4,mass effect: legendary edition,2021,Sci-Fi,9.7,706,5,1996.0,11.27,8.89,10.22,1.00,31.37
5,god of war,2018,"Action, Adventure, Drama",9.6,21310,6,1989.0,23.20,2.26,4.22,0.58,30.26
6,perusona 5: za roiyaru,2019,"Action, Adventure, Fantasy",9.6,798,7,2006.0,11.38,9.23,6.50,2.90,30.01
7,zeruda no densetsu: toki no okarina,1998,"Action, Adventure, Fantasy",9.6,8998,8,2006.0,14.03,9.20,2.93,2.85,29.02
8,metal gear solid,1998,"Action, Adventure, Drama",9.6,11329,9,2009.0,14.59,7.06,4.70,2.26,28.62
9,wiedzmin 3: dziki gon - serca z kamienia,2015,"Action, Adventure, Drama",9.5,5679,10,1984.0,26.93,0.63,0.28,0.47,28.31
